In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
url = 'https://www.dotproperty.com.my/blog/property-news/page/79'
r = requests.get(url)

In [3]:
soup = BeautifulSoup(r.content, "lxml")

In [5]:
a = soup.find_all('h3', {'class': 'entry-title td-module-title'})
a = [a_.find('a').get('href') for a_ in a]
a

['https://www.dotproperty.com.my/blog/safari-living-at-its-best',
 'https://www.dotproperty.com.my/blog/hk-and-china-2016-forecast',
 'https://www.dotproperty.com.my/blog/new-year-tips-for-landlords',
 'https://www.dotproperty.com.my/blog/london-tops-hk-for-office-prices',
 'https://www.dotproperty.com.my/blog/investors-look-to-uganda',
 'https://www.dotproperty.com.my/blog/pay-to-buy-in-australia',
 'https://www.dotproperty.com.my/blog/cars-gems-beat-property',
 'https://www.dotproperty.com.my/blog/copy-of-more-crowdfunders-for-coassets',
 'https://www.dotproperty.com.my/blog/copy-of-global-house-price-boom-grows']

In [7]:
def generator():
    while True:
        yield
        
index = 1
all_a = []

for _ in tqdm(generator()):
    url = f'https://www.dotproperty.com.my/blog/property-news/page/{index}'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")

    a = soup.find_all('h3', {'class': 'entry-title td-module-title'})
    a = [a_.find('a').get('href') for a_ in a]
    
    if not len(a):
        break
        
    all_a.extend(a)
    
    index += 1

259it [02:11,  1.97it/s]


KeyboardInterrupt: 

In [8]:
urls = sorted(list(set(all_a)))
len(urls)

789

In [9]:
urls[:10]

['https://www.dotproperty.com.my/blog/117-spike-incoming-residents-uks-best-city-property-investment',
 'https://www.dotproperty.com.my/blog/2018-budget-requests-made',
 'https://www.dotproperty.com.my/blog/2018-interior-design-trends',
 'https://www.dotproperty.com.my/blog/2019-interior-design-trends-condos',
 'https://www.dotproperty.com.my/blog/2370-2',
 'https://www.dotproperty.com.my/blog/3-benefits-starting-home-search-at-home',
 'https://www.dotproperty.com.my/blog/3-easy-ways-reduce-water-usage-home',
 'https://www.dotproperty.com.my/blog/3-interior-design-trends-southeast-asia-watch-2023',
 'https://www.dotproperty.com.my/blog/3-key-facing-philippine-property-market-2021-year',
 'https://www.dotproperty.com.my/blog/3-key-questions-facing-thailand-property-market-2021-year']

In [10]:
url = urls[0]
r = requests.get(url)
soup = BeautifulSoup(r.content, "lxml")

In [11]:
soup.find('h1').text.strip()

'With a 117% spike in incoming residents, here is the UK’s best city for property investment'

In [13]:
soup.find('div', {'class': 'td-post-content'}).get_text(separator="\n")

"\n\nThe Crown Street neighbourhood near the Manchester City Centre has become one of the city's most coveted places to live\n\n\nThe UK property market has revolved around London for seemingly ever. Obviously, its status as a population and financial hub has provided the capital with a relatively stable base. However, there has been a movement away from London in recent times with other cities taking advantage of the situation.\n\n\nPerhaps no city has benefited from the shift away from London than Manchester. The city is probably best known in Asia as being the home to football powerhouses Manchester United and Manchester City, but the performance of the local property market surpasses just about anything seen on the pitch.\n\n\nAffordability along with employment growth and sustained inward investment has driven an influx of people to the city. In July alone, JLL reported Manchester recorded a 117% increase in people moving to the city year-on-year. And this isn’t a short-term gain.

In [16]:
data = []
for url in tqdm(urls):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    try:
        data.append({
            'url': url,
            'title': soup.find('h1').text.strip(),
            'body': soup.find('div', {'class': 'td-post-content'}).get_text(separator="\n"),
        })
    except:
        pass

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 789/789 [06:45<00:00,  1.94it/s]


In [17]:
data[0]

{'url': 'https://www.dotproperty.com.my/blog/117-spike-incoming-residents-uks-best-city-property-investment',
 'title': 'With a 117% spike in incoming residents, here is the UK’s best city for property investment',
 'body': "\n\nThe Crown Street neighbourhood near the Manchester City Centre has become one of the city's most coveted places to live\n\n\nThe UK property market has revolved around London for seemingly ever. Obviously, its status as a population and financial hub has provided the capital with a relatively stable base. However, there has been a movement away from London in recent times with other cities taking advantage of the situation.\n\n\nPerhaps no city has benefited from the shift away from London than Manchester. The city is probably best known in Asia as being the home to football powerhouses Manchester United and Manchester City, but the performance of the local property market surpasses just about anything seen on the pitch.\n\n\nAffordability along with employment

In [18]:
with open('dotproperty.com.my.jsonl', 'w') as fopen:
    for d in data:
        fopen.write(f'{json.dumps(d)}\n')